In [8]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

In [9]:
class KeypointsDataset(Dataset):
    def __init__(self, inputs, labels, transform=None):
      self.inputs = inputs
      self.labels = labels
      self.transform = transform

    def __len__(self):
      return len(self.inputs)

    def __getitem__(self, index):
      label = torch.tensor(self.labels[index],dtype=torch.float32)
      input = torch.tensor(self.inputs[index],dtype=torch.float32)
      return input,label

In [10]:
import pickle
with open('/content/pose_track_with_ball_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [11]:
data = np.array(data)

In [12]:
labels_data = data[:,53]

In [13]:
input_data = np.concatenate((data[:, :53], data[:, 54:]), axis=1) #Accidentally saved the labels not on the last index in each row so have to extract input like this
labels_data = data[:,53]

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

In [15]:
pipeline = make_pipeline(Normalizer(),StandardScaler()) 

In [16]:
input_data = pipeline.fit_transform(input_data)

In [17]:
from joblib import dump, load
dump(pipeline, 'pipeline.pkl') 

['pipeline.pkl']

In [18]:
class ReversibilityClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        input_size = 57
        self.fc1 = nn.Linear(in_features=input_size, out_features=2*input_size)
        self.fc2 = nn.Linear(in_features=2*input_size, out_features=input_size)
        self.fc3 = nn.Linear(in_features=input_size, out_features=1)
        self.reLU = nn.ReLU()
        self.dropout = nn.Dropout(p=0.4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.reLU(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.reLU(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [19]:
import math
train_size = math.floor(0.7 * len(data))
test_size = len(data) - train_size

pose_dataset = KeypointsDataset(input_data, labels_data)

train_dataset, test_dataset = random_split(pose_dataset, [train_size,test_size])

train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
testing_loader = DataLoader(test_dataset, batch_size=1000, shuffle=True)

In [20]:
model = ReversibilityClassifier()
device = torch.device("cuda")
model.to(device)

ReversibilityClassifier(
  (fc1): Linear(in_features=57, out_features=114, bias=True)
  (fc2): Linear(in_features=114, out_features=57, bias=True)
  (fc3): Linear(in_features=57, out_features=1, bias=True)
  (reLU): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (sigmoid): Sigmoid()
)

In [88]:
model.train()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


n_epochs = 400
for epoch in range(n_epochs):
    total_loss = 0
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        targets = targets.unsqueeze(1)
        loss = criterion(outputs, targets)

        total_loss += loss.detach().item()
        loss.backward()
        optimizer.step()
    average_loss = total_loss / len(train_loader)
    print(f"Average loss for epoch {epoch}: {average_loss:.2f}")

Average loss for epoch 0: 0.71
Average loss for epoch 1: 0.69
Average loss for epoch 2: 0.69
Average loss for epoch 3: 0.69
Average loss for epoch 4: 0.69
Average loss for epoch 5: 0.69
Average loss for epoch 6: 0.69
Average loss for epoch 7: 0.68
Average loss for epoch 8: 0.68
Average loss for epoch 9: 0.68
Average loss for epoch 10: 0.67
Average loss for epoch 11: 0.67
Average loss for epoch 12: 0.66
Average loss for epoch 13: 0.65
Average loss for epoch 14: 0.64
Average loss for epoch 15: 0.64
Average loss for epoch 16: 0.63
Average loss for epoch 17: 0.63
Average loss for epoch 18: 0.62
Average loss for epoch 19: 0.61
Average loss for epoch 20: 0.61
Average loss for epoch 21: 0.61
Average loss for epoch 22: 0.60
Average loss for epoch 23: 0.60
Average loss for epoch 24: 0.59
Average loss for epoch 25: 0.59
Average loss for epoch 26: 0.59
Average loss for epoch 27: 0.59
Average loss for epoch 28: 0.58
Average loss for epoch 29: 0.58
Average loss for epoch 30: 0.58
Average loss for e

In [106]:
model.eval()
correct = 0
total = 0
all_predictions = []
all_labels = []
for test_inp, test_labels in testing_loader:
  test_inp,test_labels = test_inp.cuda(),test_labels.cuda()
  output = model(test_inp)
  predictions = []

  for i in range(len(output)):
    if output[i] >= 0.5:
      predictions.append(1)
    else:
      predictions.append(0)
    test_labels = test_labels.cpu()  
  all_predictions = all_predictions + predictions
  all_labels = all_labels + test_labels.tolist()
  correct += (np.array(predictions)==np.array(test_labels)).sum()
  total += float(test_labels.size(0))

print (correct)
print (total)
print ((correct/total)*100)

32244
40762.0
79.1030862077425


In [115]:
torch.save(model.state_dict(),"final_model.pth")